In [4]:
%%capture
from pathlib import Path
import pandas as pd
import numpy as np

In [5]:
DATA = Path.cwd().parent / 'data'
SUMM_FOLDER = DATA / 'summaries_finetune'
TEXT_FILES = SUMM_FOLDER / 'text_files_copy'
SOURCE_TEXTS = SUMM_FOLDER / 'source_texts_clean'

In [6]:
p = Path(TEXT_FILES).rglob('*')
file_dict = {'file_path': [x for x in p if x.is_file()]}
file_dict['file_name'] = [str(x).split('/')[-1] for x in file_dict['file_path']]

In [7]:
file_df = pd.DataFrame.from_dict(file_dict).set_index('file_name')

In [8]:
# The original file had 500 empty columns. Don't need those
summaries_df = pd.read_csv(SUMM_FOLDER / 'final_summaries_ai_aloe.csv').iloc[:,0:14]

# Here's a function to get the text. I'm ignoring encoding errors for now, but I need to figure that out.
def getText(filename):
    if filename in file_df.index:
        with open(file_df.loc[filename, 'file_path'], 'r',encoding='utf-8', errors='ignore') as f:
            text = f.read()
        return text
    else:
        return np.nan

# Apply the function
summaries_df['text'] = summaries_df['filename'].apply(lambda x: getText(x))

In [9]:
summaries_df['text'].isna().sum()

0

## Get the Source Texts in there

In [19]:
p = Path(SOURCE_TEXTS).rglob('*')
file_dict = {'file_path': [x for x in p if x.is_file()]}
file_dict['file_name'] = [str(x).split('/')[-1] for x in file_dict['file_path']]
file_df = pd.DataFrame.from_dict(file_dict)
file_df['source_text_clean'] = file_df['file_name'].apply(lambda x: x.split('.')[0])
file_df = file_df.set_index('source_text_clean')
file_df

,file_path,file_name
source_text_clean,,
12_JobSearch,/home/jovyan/shared/2022_09_21_textbook/data/s...,12_JobSearch.txt
Electric Circuits,/home/jovyan/shared/2022_09_21_textbook/data/s...,Electric Circuits.txt
28_CarPollution,/home/jovyan/shared/2022_09_21_textbook/data/s...,28_CarPollution.txt
Alzheimer's_Family,/home/jovyan/shared/2022_09_21_textbook/data/s...,Alzheimer's_Family.txt
Microbes - Advanced_c5cbea4e-07be-4ba9-ae07-02a348c5fb02,/home/jovyan/shared/2022_09_21_textbook/data/s...,Microbes - Advanced_c5cbea4e-07be-4ba9-ae07-02...
...,...,...
Food Webs - Easy_fb410417-58c9-4ec8-93f4-2e51bed70d21,/home/jovyan/shared/2022_09_21_textbook/data/s...,Food Webs - Easy_fb410417-58c9-4ec8-93f4-2e51b...
Red Blood Cells SE Task,/home/jovyan/shared/2022_09_21_textbook/data/s...,Red Blood Cells SE Task.txt
20_Disabilities,/home/jovyan/shared/2022_09_21_textbook/data/s...,20_Disabilities.txt


In [11]:
summaries_df['source_text_clean'] = summaries_df['source_text'].apply(lambda x: x.split('.')[0])
summaries_df

,Unnamed: 0,Row.names,filename,filename_clean,source_text,Main.Point,Details,Cohesion,Objective.Language,Wording.Para,Lang..Bey..ST,Summ..Length,content_pca,paraphrase_pca,text,source_text_clean
0,1,1,1091_CivilServices .txt,1091_CivilServices,CivilService.txt,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1.37,0.785,hard work pays off / / \n,CivilService
1,2,10,455436_Hybrids .txt,455436_Hybrids,Hybrids.txt,0.5,0.5,0.5,0.5,0.0,0.5,0.5,1.37,0.245,There will be a more petroleum product coming...,Hybrids
2,3,100,355927_SexDetermination .txt,355927_SexDetermination,Sex Determination.txt,1.0,2.0,3.0,1.5,0.5,1.0,1.0,5.60,1.030,Males have one Y chromosome and one X chromoso...,Sex Determination
3,4,1000,MDSum_A_Sum_037_Text1.txt.txt,MDSum_A_Sum_037_Text1,sun_exposure,2.5,2.0,1.5,2.0,2.5,1.5,1.0,5.42,3.435,Electromagnetic radiation with an energy and s...,sun_exposure
4,5,1001,MDSum_A_Sum_037_Text2.txt.txt,MDSum_A_Sum_037_Text2,sun_exposure,2.5,2.0,2.0,2.5,3.0,2.0,1.0,5.86,4.220,It is highly recommended by doctors that peopl...,sun_exposure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,4686,995,MDSum_A_Sum_029_Text2.txt.txt,MDSum_A_Sum_029_Text2,sun_exposure,2.5,2.0,2.5,2.5,1.0,2.0,2.5,6.30,2.060,The results from many studies indicate that vi...,sun_exposure
4686,4687,996,MDSum_A_Sum_029_Text4.txt.txt,MDSum_A_Sum_029_Text4,sun_exposure,2.5,2.0,3.0,2.0,1.0,2.0,1.5,6.74,2.060,People are being advised to spend less time in...,sun_exposure
4687,4688,997,MDSum_A_Sum_029_Text5.txt.txt,MDSum_A_Sum_029_Text5,sun_exposure,2.5,2.5,2.0,2.5,1.0,2.0,1.0,6.41,2.060,We are thus in a situation where people are re...,sun_exposure
4688,4689,998,MDSum_A_Sum_031_Text3.txt.txt,MDSum_A_Sum_031_Text3,sun_exposure,2.5,3.0,3.5,2.0,2.5,2.5,3.0,8.28,3.925,"There are two types of cancers, melanoma and b...",sun_exposure


### The prompt names in the csv are wrong. Changing them.

In [50]:
prompt_names_df = pd.read_csv('prompt_names.csv').set_index('csvname_clean')
prompt_names_df.loc['Hybrids', 'filename_clean']

'24_Hybrids.txt'

In [53]:
summaries_df['source_text_filename_clean'] = summaries_df['source_text_clean'].apply(lambda x: prompt_names_df.loc[x, 'filename_clean'])
summaries_df['source_text_filename_clean'] = summaries_df['source_text_filename_clean'].apply(lambda x: '24_Hybrids' if x == '24_Hybrids.txt' else x)


In [54]:
def getText(filename):
    if filename in file_df.index:
        with open(file_df.loc[filename, 'file_path'], 'r',encoding='utf-8', errors='ignore') as f:
            text = f.read()
        return text
    else:
        return np.nan
    
summaries_df['prompt'] = summaries_df['source_text_filename_clean'].apply(lambda x: getText(x))
summaries_df[summaries_df['prompt'].isna()]

,Unnamed: 0,Row.names,filename,filename_clean,source_text,Main.Point,Details,Cohesion,Objective.Language,Wording.Para,Lang..Bey..ST,Summ..Length,content_pca,paraphrase_pca,text,source_text_clean,source_text_filename_clean,prompt


In [57]:
summaries_df.to_csv('final_summaries_ai_aloe_fixed.csv')